## Define directory paths to images and labels

In [1]:
# Replace the path to the folder in the string
# (Copy path directly from the windows explorer/ mac finder)
images_dir_path = r"C:\Users\Lakshya Dev\Downloads\Work\Workspace\Bosch Hackathon\labelled_data\sanjana\set_1"
# Replace the path to the folder in the string
# (Copy path directly from the windows explorer/ mac finder)
labels_dir_path = r"C:\Users\Lakshya Dev\Downloads\Work\Workspace\Bosch Hackathon\labelled_data\sanjana\Labelled Img"


---
# Pretreat Images & Labels Script
  ----------------------
  ----------------------

## Imports

In [2]:
# Importing all necessary libraries
from os import walk
import os
from os.path import join as joinPath
import pandas as pd
import numpy as np

---
## Make sure the images and labels match

In [3]:
from match_labels_with_images import match_labels_with_images

match_labels_with_images(images_dir_path=images_dir_path, labels_dir_path=labels_dir_path, printFullLogs=False)

Running match_labels_with_images script ...
results {'images_dir_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\set_1', 'labels_dir_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\Labelled Img', 'mismatch_count': 17, 'mismatched_images': ['frame220.jpg', 'frame587.jpg', 'frame524.jpg', 'frame322.jpg', 'frame320.jpg', 'frame321.jpg', 'frame319.jpg', 'frame523.jpg', 'frame291.jpg', 'frame525.jpg', 'frame586.jpg', 'frame292.jpg', 'frame406.jpg', 'frame318.jpg', 'frame290.jpg', 'frame377.jpg', 'frame526.jpg'], 'mismatched_images_archive_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\set_1\\mismatched_images_archive'}
Completed running match_labels_with_images script ...


{'images_dir_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\set_1',
 'labels_dir_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\Labelled Img',
 'mismatch_count': 17,
 'mismatched_images': ['frame220.jpg',
  'frame587.jpg',
  'frame524.jpg',
  'frame322.jpg',
  'frame320.jpg',
  'frame321.jpg',
  'frame319.jpg',
  'frame523.jpg',
  'frame291.jpg',
  'frame525.jpg',
  'frame586.jpg',
  'frame292.jpg',
  'frame406.jpg',
  'frame318.jpg',
  'frame290.jpg',
  'frame377.jpg',
  'frame526.jpg'],
 'mismatched_images_archive_path': 'C:\\Users\\Lakshya Dev\\Downloads\\Work\\Workspace\\Bosch Hackathon\\labelled_data\\sanjana\\set_1\\mismatched_images_archive'}

---
## Classes and labels data for treatment

### 1. Define predefined classes to treat

In [4]:
target_labels = {
    "Furniture" : 0,
    "Door": 1,
    "Small Garments": 2,
    "Cables": 3
}
target_labels.keys()

dict_keys(['Furniture', 'Door', 'Small Garments', 'Cables'])

### 2. Load classes and labels

In [5]:

# imgDirPath, _, imgFilenames = next(walk(images_dir_path))
# imgFilenames = [name for name in set(imgFilenames) if (name.endswith(".jpg"))]
lblDirPath, _, lblFilenames = next(walk(labels_dir_path))

classes = pd.read_table(joinPath(lblDirPath,lblFilenames.pop(0)), header=None, names=['class'])
        
lblFilenames = list(set([name for name in set(lblFilenames) if (name.endswith(".txt"))]))

current_labels = {}
for index in classes.index:
    label = classes["class"][index]
    if (label in target_labels.keys()): 
        current_labels[index] = label

current_labels = dict(current_labels)

### 3. Save new classes file

In [6]:
corrected_labels_dir = joinPath(lblDirPath,'corrected_labels')
if not os.path.exists(corrected_labels_dir):
                    os.makedirs(corrected_labels_dir)
updated_classes = pd.DataFrame(["Furniture", "Door", "Small Garments", "Cables"])
np.savetxt(joinPath(corrected_labels_dir,"classes.txt"), updated_classes.values, fmt='%s')

### 4. Iterate fr each label file and treat

In [7]:
error_files = []
errors = []
for lblFile in lblFilenames:
    print(f'Correcting labels in file {lblFile} ...')
    objects = pd.read_table(joinPath(lblDirPath,lblFile), names=['anotations'])
    updated_objects = []
    for obj in objects.anotations:
        try:
            obj_items = obj.split(maxsplit=1)
            obj_lbl = int(obj_items[0])
            obj_items[0] = str(target_labels[current_labels[obj_lbl]])
            updated_objects.append(' '.join(obj_items))
        except Exception as e:
            error_files.append(lblFile)
            error = f'Error in file {lblFile}-> Label not found {e}'
            errors.append(error)
            print(error)
    if len(updated_objects) > 0:            
        updated_objects = pd.DataFrame(updated_objects)
        np.savetxt(joinPath(corrected_labels_dir,lblFile), updated_objects.values, fmt='%s')
    

txt-> Label not found 2
Correcting labels in file frame401.txt ...
Correcting labels in file frame403.txt ...
Correcting labels in file frame64.txt ...
Error in file frame64.txt-> Label not found 2
Correcting labels in file frame591.txt ...
Correcting labels in file frame17.txt ...
Error in file frame17.txt-> Label not found 2
Correcting labels in file frame668.txt ...
Correcting labels in file frame444.txt ...
Correcting labels in file frame116.txt ...
Error in file frame116.txt-> Label not found 2
Correcting labels in file frame4.txt ...
Correcting labels in file frame271.txt ...
Correcting labels in file frame429.txt ...
Correcting labels in file frame645.txt ...
Correcting labels in file frame621.txt ...
Error in file frame621.txt-> Label not found 4
Correcting labels in file frame455.txt ...
Correcting labels in file frame240.txt ...
Correcting labels in file frame393.txt ...
Error in file frame393.txt-> Label not found 2
Correcting labels in file frame691.txt ...
Correcting label

### 5. Save error files

In [8]:
error_labels_dir = joinPath(lblDirPath,'error_files')
if not os.path.exists(error_labels_dir):
                    os.makedirs(error_labels_dir)
np.savetxt(joinPath(error_labels_dir,"error_files_names.txt"), pd.DataFrame(error_files).values, fmt='%s')
np.savetxt(joinPath(error_labels_dir,"error_logs.txt"), pd.DataFrame(errors).values, fmt='%s')
print(f'Saved error filenames and logs')
for error_file in set(error_files):
    os.rename(joinPath(lblDirPath, error_file), joinPath(error_labels_dir, error_file))
    print(f'Moving file {error_file} to error_files/ directory')

Saved error filenames and logs
Moving file frame64.txt to error_files/ directory
Moving file frame73.txt to error_files/ directory
Moving file frame17.txt to error_files/ directory
Moving file frame12.txt to error_files/ directory
Moving file frame116.txt to error_files/ directory
Moving file frame96.txt to error_files/ directory
Moving file frame637.txt to error_files/ directory
Moving file frame207.txt to error_files/ directory
Moving file frame621.txt to error_files/ directory
Moving file frame393.txt to error_files/ directory
Moving file frame305.txt to error_files/ directory
Moving file frame607.txt to error_files/ directory
Moving file frame399.txt to error_files/ directory
Moving file frame606.txt to error_files/ directory
Moving file frame674.txt to error_files/ directory
Moving file frame367.txt to error_files/ directory
Moving file frame211.txt to error_files/ directory
Moving file frame67.txt to error_files/ directory
Moving file frame425.txt to error_files/ directory
Moving